<a href="https://colab.research.google.com/github/leomarfmn/a-simple-sql-project/blob/main/1_DATA_COLLECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A SIMPLE SQL PROJECT** (1. DATA COLLECTION)

---



### **1.1 IMPORTS**

In [1]:
import pandas as pd               #dataframe manipulation
import numpy as np                #creating Nan values
import json                       #parsing and reading json
import requests                   #API access
import time                       #time.sleep on API loop

### **1.2 UPLOAD SPOTIFY FILE INTO NOTEBOOK**
In this section, the streaming history file from Spotify is loaded and transformed into a pandas dataframe. Only the history from March, April and May 2022 is kept, since the API requests are limited, as explained in the README file. Also, the extra columns for data expansion are created.

In [2]:
filename = '/content/StreamingHistory0.json'
with open(filename) as f:
   data = json.load(f)

In [3]:
df = pd.DataFrame(columns = ['endTime','artistName', 'trackName', 'msPlayed'])
for i in range(len(data)):
  df.loc[len(df.index)] = data[i].values()

In [4]:
df['endTime'] = pd.to_datetime(df['endTime'])
df = df[(df['endTime'].dt.to_period('M') == '2022-05') | (df['endTime'].dt.to_period('M') == '2022-04') | (df['endTime'].dt.to_period('M') == '2022-03')]

In [ ]:
df['trackID'] = ''
df['albumID'] = ''
df['genre'] = ''
df['duration'] = ''
df['albumName'] = ''
df['artistID'] = ''
df.reset_index(inplace=True, drop=True)

### **1.3 FUNCTION TO CREATE API REQUESTS**
The API requests and daily limit (quota) management were performed using RapidAPI. The key used was omitted for safety purposes and the indexes passed as arguments to the function were used for free quota control, but are not mandatory.


In [12]:
def api_request(df, initial_index, final_index):

  url = "https://theaudiodb.p.rapidapi.com/searchtrack.php"
  headers = {
  "X-RapidAPI-Key": "", # Insert key here
  "X-RapidAPI-Host": "theaudiodb.p.rapidapi.com"
  }

  if final_index > projectfile.shape[0]:
    final_index = projectfile.shape[0]

  for i in range(initial_index,final_index,1):

    # BUILD SEARCH PARAMETERS
    querystring = {"s": df.loc[i,'artistName'], "t": df.loc[i,'trackName']}
    print(querystring)


    # TAKE BREAK BETWEEN REQUESTS
    time.sleep(3)


    # API REQUEST AND RESPONSE FORMATTING
    response = requests.request("GET", url, headers=headers, params=querystring)
    parsed_response = json.loads(response.text)


    # SAVE DATA FROM RESPONSE
    try:
      df.loc[i,'trackID'] = parsed_response['track'][0]['idTrack']
    except:
      df.loc[i,'trackID'] = np.nan
      
    try:
      df.loc[i,'albumID'] = parsed_response['track'][0]['idAlbum']
    except:
      df.loc[i,'albumID'] = np.nan
      
    try:
      df.loc[i,'genre'] = parsed_response['track'][0]['strGenre']
    except:
      df.loc[i,'genre'] = np.nan
      
    try:
      df.loc[i,'duration'] = parsed_response['track'][0]['intDuration']
    except:
      df.loc[i,'duration'] = np.nan
      
    try:
      df.loc[i,'albumName'] = parsed_response['track'][0]['strAlbum']
    except:
      df.loc[i,'albumName'] = np.nan
      
    try:
      df.loc[i,'artistID'] = parsed_response['track'][0]['idArtist']
    except:
      df.loc[i,'artistID'] = np.nan
      

### **1.4 FIGURE WHERE TO START REQUESTS ON THE DATA FRAME**
Used for quota control.

In [7]:
# When using quota control, one might need to perform batch API requests on multiple days. And, because of that, one will also need to save and reuse files, as well as
# keeping track of where to begin requests the next day. The process of reinserting .csv files is straightforward, however, empty cells are also filled with 'Nan" when
# the files are saved. The lines of code below constitute a simple method of substituting non-desired 'Nan' files with empty strings in order to figure out which data 
# should then be requested.

projectfile = pd.concat([df[:634],df[634:].fillna('')])                    #634 should be read as placeholder
initial_index = projectfile.loc[projectfile['trackID'] == ''].index[0]     #initial_index is the first row where trackID equals to an empty string
final_index = initial_index + 98                                           #final_index is the initial_index plus a chosen number
print(initial_index, final_index)

0 98


In [ ]:
# When not using quota control:
'''projectfile = df
initial_index = 0
final_index = projecfile.shape[0]'''

### **1.5 SEND API REQUESTS**




In [13]:
api_request(projectfile, initial_index, final_index)

{'s': 'Nirvana', 't': 'Lithium'}
{'s': 'The Killers', 't': 'Mr. Brightside'}
{'s': 'Arctic Monkeys', 't': 'When The Sun Goes Down'}
{'s': 'Arctic Monkeys', 't': "Why'd You Only Call Me When You're High?"}
{'s': 'Arctic Monkeys', 't': 'Do I Wanna Know?'}


### **1.6 CHECK DATAFRAME AND SAVE FILE**
Checks file after API requests and saves it to a new file, so it can be used on "2. SQL DATABASE"

In [15]:
projectfile

,Unnamed: 0,endTime,artistName,trackName,trackID,albumID,genre,duration,albumName,artistID
0,0,2022-03-02 20:52:00,Eminem,Lose Yourself,32925676.0,2126328.0,Hip-Hop,326466.0,Curtain Call: The Hits,111304.0
1,1,2022-03-02 20:52:00,Eminem,Lose Yourself,32925676.0,2126328.0,Hip-Hop,326466.0,Curtain Call: The Hits,111304.0
2,2,2022-03-02 20:55:00,Piso 21,Puntos Suspensivos,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2022-03-02 21:48:00,Piso 21,Me Llamas,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2022-03-02 21:59:00,Piso 21,Puntos Suspensivos,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
677,677,2022-05-31 19:10:00,Nirvana,Lithium,32736471.0,2110839.0,Grunge,256000.0,Nevermind,111319.0
678,678,2022-05-31 19:13:00,The Killers,Mr. Brightside,33550031.0,2177412.0,Indie,222586.0,Hot Fuss,112016.0
679,679,2022-05-31 19:16:00,Arctic Monkeys,When The Sun Goes Down,32769407.0,2113920.0,Indie,200000.0,"Whatever People Say I Am, That's What I'm Not",111644.0
680,680,2022-05-31 19:16:00,Arctic Monkeys,Why'd You Only Call Me When You're High?,33584144.0,2180196.0,Indie,161000.0,AM,111644.0


In [ ]:
projectfile.to_csv('fileversion1.csv')